In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install requests -q

In [3]:
import requests
import json

In [17]:
import requests
from kaggle_secrets import UserSecretsClient

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Elevate-Program")


# ✅ UPDATED to new Hugging Face router endpoints
SENTIMENT_URL = "https://router.huggingface.co/hf-inference/models/cardiffnlp/twitter-roberta-base-sentiment-latest"
EMOTION_URL   = "https://router.huggingface.co/hf-inference/models/j-hartmann/emotion-english-distilroberta-base"
HEADERS = {"Authorization": f"Bearer {HF_API_TOKEN}"}

print("✅ Setup complete!")


✅ Setup complete!


In [18]:
def analyze_sentiment(text):
    response = requests.post(SENTIMENT_URL, headers=HEADERS, json={"inputs": text})
    results  = sorted(response.json()[0], key=lambda x: x['score'], reverse=True)

    label_map = {
        "LABEL_0": "NEGATIVE",
        "LABEL_1": "NEUTRAL",
        "LABEL_2": "POSITIVE"
    }

    print("\n" + "="*55)
    print(f"  Text      : {text}")
    print(f"  Sentiment : {label_map.get(results[0]['label'], results[0]['label'])}")
    print("-"*55)
    for r in results:
        label = label_map.get(r['label'], r['label'])
        score = round(r['score'] * 100, 2)
        bar   = "█" * int(score / 5)
        print(f"  {label:<12}: {score:>6}%  {bar}")
    print("="*55)
    return label_map.get(results[0]['label'], results[0]['label'])


In [19]:
def detect_emotion(text):
    response = requests.post(EMOTION_URL, headers=HEADERS, json={"inputs": text})
    emotions = sorted(response.json()[0], key=lambda x: x['score'], reverse=True)
    top      = emotions[0]
    print(f"  Emotion   : {top['label'].upper()} ({round(top['score']*100, 2)}%)")

In [20]:
def analyze_sentiment(text):
    response = requests.post(SENTIMENT_URL, headers=HEADERS, json={"inputs": text})
    data = response.json()

    # Debug: print raw response if error
    if isinstance(data, dict) and "error" in data:
        print(f"  API Error: {data['error']}")
        return

    results = sorted(data[0], key=lambda x: x['score'], reverse=True)

    # FIXED: Handle all possible label formats the model may return
    def clean_label(label):
        label = label.upper()
        if label in ["LABEL_0", "NEG", "NEGATIVE"]:
            return "NEGATIVE"
        elif label in ["LABEL_1", "NEU", "NEUTRAL"]:
            return "NEUTRAL"
        elif label in ["LABEL_2", "POS", "POSITIVE"]:
            return "POSITIVE"
        return label  # return as-is if unknown

    print("\n" + "="*55)
    print(f"  Text      : {text}")
    print(f"  Sentiment : {clean_label(results[0]['label'])}")
    print("-"*55)
    for r in results:
        label = clean_label(r['label'])
        score = round(r['score'] * 100, 2)
        bar   = "█" * int(score / 5)
        print(f"  {label:<12}: {score:>6}%  {bar}")
    print("="*55)
    return clean_label(results[0]['label'])


In [21]:
def detect_emotion(text):
    response = requests.post(EMOTION_URL, headers=HEADERS, json={"inputs": text})
    data = response.json()

    if isinstance(data, dict) and "error" in data:
        print(f"  Emotion API Error: {data['error']}")
        return

    emotions = sorted(data[0], key=lambda x: x['score'], reverse=True)
    top = emotions[0]
    print(f"  Emotion   : {top['label'].upper()} ({round(top['score']*100, 2)}%)")

In [22]:
sample_texts = [
    "I absolutely love Microsoft Azure and AI technologies!",
    "The service was just okay, nothing special about it.",
    "This is the worst experience I have ever had. Terrible!",
    "Hugging Face makes AI development very easy and accessible.",
    "I am not sure whether I like this product or not."
]

print("\n📊 SENTIMENT ANALYSIS RESULTS")
print("   Azure Elevate Program | Edunet Foundation & AICTE\n")

for text in sample_texts:
    analyze_sentiment(text)
    detect_emotion(text)


📊 SENTIMENT ANALYSIS RESULTS
   Azure Elevate Program | Edunet Foundation & AICTE


  Text      : I absolutely love Microsoft Azure and AI technologies!
  Sentiment : POSITIVE
-------------------------------------------------------
  POSITIVE    :  99.05%  ███████████████████
  NEUTRAL     :   0.67%  
  NEGATIVE    :   0.29%  
  Emotion   : JOY (97.91%)

  Text      : The service was just okay, nothing special about it.
  Sentiment : NEUTRAL
-------------------------------------------------------
  NEUTRAL     :  47.46%  █████████
  POSITIVE    :  27.24%  █████
  NEGATIVE    :   25.3%  █████
  Emotion   : NEUTRAL (83.73%)

  Text      : This is the worst experience I have ever had. Terrible!
  Sentiment : NEGATIVE
-------------------------------------------------------
  NEGATIVE    :   95.4%  ███████████████████
  NEUTRAL     :   3.84%  
  POSITIVE    :   0.76%  
  Emotion   : DISGUST (88.63%)

  Text      : Hugging Face makes AI development very easy and accessible.
  Sentiment : PO

In [23]:
my_text = "Microsoft Azure Elevate program by Edunet and AICTE is amazing!"

print("\n🔍 CUSTOM TEXT ANALYSIS")
analyze_sentiment(my_text)
detect_emotion(my_text)


🔍 CUSTOM TEXT ANALYSIS

  Text      : Microsoft Azure Elevate program by Edunet and AICTE is amazing!
  Sentiment : POSITIVE
-------------------------------------------------------
  POSITIVE    :  98.54%  ███████████████████
  NEUTRAL     :   1.24%  
  NEGATIVE    :   0.21%  
  Emotion   : SURPRISE (71.54%)
